In [ ]:
# MY CODE FOR TESTING AND UNDERSTANDING HOW IT WORKS

In [ ]:
! pip install chess
! pip install atomic

import argparse
import chess
import encoder
import MCTS
import torch
import AlphaZeroNetwork
import time

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.1/149.1 kB 2.5 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for atomic: filename=atomic-0.7.3-cp39-cp39-linux_x86_64.whl size=26998 sha256=c6a9b4ef3526466bfff107f44acf756293c5e1eba053062414f4a366d6bdca78
  Stored in directory: /root/.cache/pip/wheels/d4/25/e6/a30e521556b4fc888653029fc756f94e21c613fee6fddffcd1
Successfully built atomic


In [3]:

def tolist( move_generator ):
    """
    Change an iterable object of moves to a list of moves.
    
    Args:
        move_generator (Mainline object) iterable list of moves

    Returns:
        moves (list of chess.Move) list version of the input moves
    """
    moves = []
    for move in move_generator:
        moves.append( move )
    return moves


In [4]:
alphaZeroNet = AlphaZeroNetwork.AlphaZeroNet(20, 256)
weights = torch.load("jaja.pt")
alphaZeroNet.load_state_dict( weights )
alphaZeroNet = alphaZeroNet.cuda()
for param in alphaZeroNet.parameters():
    param.requires_grad = False
r = alphaZeroNet.eval()

In [5]:
import numpy as np


def encodePositionForInference( board ):
    #Flip if black's turn
    if not board.turn:
        board = board.mirror()
    positionPlanes = encoder.encodePosition( board )
    mask = encoder.getLegalMoveMask( board )
    return positionPlanes, mask

board = chess.Board()
position, mask = encodePositionForInference(board)
position = torch.from_numpy( position )[ None, ... ]
mask = torch.from_numpy( mask )[ None, ... ]
if True:
    position = position.cuda()
    mask = mask.cuda()
# value, policy = alphaZeroNet( position, policyMask=mask )

# policy = policy.cpu().numpy()[ 0 ] 

In [6]:
x = alphaZeroNet.convBlock1( position )
for block in alphaZeroNet.residualBlocks:
    x = block( x )
value = alphaZeroNet.valueHead( x )
policy = alphaZeroNet.policyHead( x )

In [7]:
import CCRLDataset

train_ds = CCRLDataset.CCRLDataset("reformat")

In [8]:
from torch.nn import CrossEntropyLoss
loss = CrossEntropyLoss()

data = train_ds[0]

position = data['position'].cuda() # The encoded position of the game the AI can understand
valueTarget = data['value'].cuda() # Winner of game
policyTarget = data['policy'].cuda() # Index of the encoded target move




In [21]:
policyTarget = policyTarget.view( policyTarget.shape[0] )
g = loss(policy, policyTarget)


In [25]:
def cross_entropy_loss(yHat, y):
    if y == 1:
      return -np.log(yHat)
    else:
      return -np.log(1 - yHat)
    
print(cross_entropy_loss(policy.cpu().numpy(), policyTarget.cpu().numpy()))
print(g)

[[-3.1620655 -3.1350374 -3.1433704 ... -3.1563902 -3.1374476 -3.1345947]]
tensor(12.3909, device='cuda:0')


In [18]:
import numpy as np

def getLegalMoveMask( board ):
    """
    Returns a mask encoding the legal moves.
    
    Args:
        board (chess.Board) the chess position.

    Returns:
        mask (numpy.array (72, 8, 8) int32) the legal move mask
    """
    mask = np.zeros( (72, 8, 8), dtype=np.int32 )
    
    for move in board.legal_moves: # Goes through legal moves | e2e4
        planeIdx, rankIdx, fileIdx = encoder.moveToIdx( move )
        print(move, planeIdx)
        """
        The location in the plane specifies the start square.
        """
        mask[ planeIdx, rankIdx, fileIdx ] = 1

    return mask

In [9]:
board = chess.Board(fen="r1bqkbnr/ppp2ppp/2np4/4p3/2B1P3/3P1N2/PPP2PPP/RNBQK2R w KQkq - 0 1")
print(board)

MASK = getLegalMoveMask(board)
MASK = torch.from_numpy( MASK ).unsqueeze(0)
MASK = MASK.view( MASK.shape[0], -1 ).cuda()

board = chess.Board()
position, mask = encoder.encodePositionForInference(board)
position = torch.from_numpy( position ).unsqueeze(0).cuda()
mask = torch.from_numpy( mask )[ None, ... ].cuda()

r = alphaZeroNet(position, policyMask=mask)
x = alphaZeroNet.convBlock1( position )
for block in alphaZeroNet.residualBlocks:
    x = block( x )
value = alphaZeroNet.valueHead( x )
policy = alphaZeroNet.policyHead( x )


r . b q k b n r
p p p . . p p p
. . n p . . . .
. . . . p . . .
. . B . P . . .
. . . P . N . .
P P P . . P P P
R N B Q K . . R
c4f7 35
c4e6 34
c4a6 58
c4d5 33
c4b5 57
c4b3 41
f3g5 64
f3e5 68
f3h4 65
f3d4 69
f3d2 70
f3g1 67
h1g1 9
h1f1 10
e1e2 17
e1d2 57
e1f1 1
d1e2 33
d1d2 17
c1h6 37
c1g5 36
c1f4 35
c1e3 34
c1d2 33
b1c3 64
b1a3 68
b1d2 65
e1g1 2
d3d4 17
h2h3 17
g2g3 17
c2c3 17
b2b3 17
a2a3 17
h2h4 18
g2g4 18
b2b4 18
a2a4 18


In [25]:
import CCRLDataset

train_ds = CCRLDataset.CCRLDataset("reformat")

train_ds[0]

: 

In [ ]:
train_ds

Conv2d(16, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [16]:
policy_exp = torch.exp( policy ) 
policy_exp *= MASK.type( torch.float32 ) # Multiplies with all the LEGAL MOVES. Only the legal moves will stay there.
policy_exp_sum = torch.sum( policy_exp, dim=1, keepdim=True )
policy_softmax = policy_exp / policy_exp_sum
poo = policy_softmax.cpu().numpy()[ 0 ] 

print(MASK.type(torch.float32), MASK)
# This has to be some softmax BS


# OK So WTF. The LEGAL MOVE mASK SO CONFUSIng. If it only presents the starting move then how would it know where it actually moved to. Also wasnt the starting move alread in the encoded positions.
def decodePolicyOutput( board, policy ):
    """
    Decode the policy output from the neural network.

    Args:
        board (chess.Board) the board
        policy (numpy.array) the policy output

    """

    move_probabilities = np.zeros( 200, dtype=np.float32 )
    num_moves = 0
    
    for idx, move in enumerate( board.legal_moves ):
        if not board.turn:
            move = encoder.mirrorMove( move )
        planeIdx, rankIdx, fileIdx = encoder.moveToIdx( move )
        moveIdx = planeIdx * 64 + rankIdx * 8 + fileIdx
        move_probabilities[ idx ] = policy[ moveIdx ]
        print("%f" % policy[moveIdx], moveIdx)
        num_moves += 1

    return move_probabilities[ :num_moves ] # Cuts off at the move amount

print(decodePolicyOutput(board, poo))


tensor([[0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0') tensor([[0, 0, 0,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.int32)
0.000105 4102
0.127524 4358
0.005173 4097
0.000166 4353
0.000226 1103
0.034740 1102
0.000067 1101
0.009837 1100
0.004208 1099
0.002853 1098
0.016045 1097
0.000525 1096
0.000104 1167
0.000045 1166
0.007916 1165
0.375690 1164
0.173510 1163
0.240591 1162
0.000614 1161
0.000060 1160
[1.05194165e-04 1.27523705e-01 5.17278677e-03 1.65607169e-04
 2.26300093e-04 3.47395651e-02 6.66991255e-05 9.83720180e-03
 4.20831842e-03 2.85328366e-03 1.60454120e-02 5.25152311e-04
 1.03816827e-04 4.47671227e-05 7.91575667e-03 3.75690430e-01
 1.73510164e-01 2.40590960e-01 6.14352466e-04 6.04516681e-05]


In [147]:
print(poo[1164])

0.37569043


In [165]:
g = list(board.legal_moves)
print(len(g))
move = g[13]
# From Square. Imagine flattening the board. Thats the index.
print(move, move.to_square, chess.square_file( move.to_square ))
for prob in decodePolicyOutput(board, policy):
    print("%f" % prob, next(g))

20
g2g4 30 6


In [33]:
# import sys
# 
# torch.set_printoptions(profile="full")
# np.set_printoptions(threshold=sys.maxsize)


0.000105 g1h3
0.127524 g1f3
0.005173 b1c3
0.000166 b1a3
0.000226 h2h3
0.034740 g2g3
0.000067 f2f3
0.009837 e2e3
0.004208 d2d3
0.002853 c2c3
0.016045 b2b3
0.000525 a2a3
0.000104 h2h4
0.000045 g2g4
0.007916 f2f4
0.375690 e2e4
0.173510 d2d4
0.240591 c2c4
0.000614 b2b4
0.000060 a2a4


(array([1.05194165e-04, 1.27523705e-01, 5.17278677e-03, 1.65607169e-04,
       2.26300093e-04, 3.47395651e-02, 6.66991255e-05, 9.83720180e-03,
       4.20831842e-03, 2.85328366e-03, 1.60454120e-02, 5.25152311e-04,
       1.03816827e-04, 4.47671227e-05, 7.91575667e-03, 3.75690430e-01,
       1.73510164e-01, 2.40590960e-01, 6.14352466e-04, 6.04516681e-05],
      dtype=float32), <LegalMoveGenerator at 0x7f843c39bd30 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>)


In [4]:
def callNeuralNetwork( board, neuralNetwork ):
    position, mask = encodePositionForInference( board ) 
    """
    Position: One hot encoding of the chess board
    Mask: A mask of all the legal moves
        Each of the 72 planes represents a different direction
            Knights: 8 
            Rook and Bishop directions with distance (64 planes)
    """
    position = torch.from_numpy( position )[ None, ... ]
    mask = torch.from_numpy( mask )[ None, ... ]
    if True:
        position = position.cuda()
        mask = mask.cuda()
    value, policy = neuralNetwork( position, policyMask=mask )
    value = value.cpu().numpy()[ 0, 0 ] # [[]] -> int | stuff like that
    policy = policy.cpu().numpy()[ 0 ] #  
    move_probabilities = encoder.decodePolicyOutput( board, policy )
    return value, move_probabilities

In [5]:
board = chess.Board()
# board.push_san("e4")
# board.push_san("e5")
# board.push_san("f1c4")
# board.push_san("a7a6")
# board.push_san("d1h5")
# board.push_san("g8f6")
# board.push_san("h5f7")


print(board)


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [54]:
    
board = chess.Board()
print(board)



r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R


In [14]:
"""
with torch.no_grad():
    root.rollout( board.copy(), alphaZeroNet )
stat, data = root.getStatisticsString() 
print(stat)
"""

"""
Words used:
Terminal node: Last one, leaf node.
UCT: Upper Confidence Bounds
    Used to balance between exploration and exploitation of the tree nodes
    Based on the number of visits to the node and the estimated value of the node


Variables used:
Edges: Edge of every board.legal_moves

The Move
Edge: ( 
        move,
        move_probability | this move's probability from the neural network
    )
    self.P # probability
    self.move # chess move
    self.child # the child node
    self.virtualLosses = 0. # Ok wtf is this

    has_child
    getN, getP, getQ
    expand
        ???
    getChild
    getMove
    addVirtualLoss
        ???
    clearVirtualLoss

Basically this state of the board.
Node: (
        board, 
        new_Q, | The prob of winning according to neural network
        move_probabilities | (numpy.array (200) float) probability distribution across move list
    )
    self.N | Visit Count | starts at 1
    self.sum_Q | Win probabilities in the subtree from POV of node
    self.edges 

"""


"""
1. Append the current node to `node_path`
2. Get the move that maximizes the UCT
    a) Add to `edge_path`
    b) If it's terminal node: 
        Break
3. Add virtual loss to the node (+1)
4. Push the chess move
5. Check if the node has a child
    a) Break if it doesn't | Cuz it hasn't been expanded yet
6. Assign the child OF the chosen node AS the new node. Why TF does this exist. Because board's doing more moves it has to update.
7. Repeat 1. using the new node. 
"""
def selection(board):
    with torch.no_grad():
        node_path = [] # list of nodes traversed 
        edge_path = [] # list of edges traversed
        root.selectTask(board, node_path, edge_path)
    return node_path, edge_path

def expansion(edge_path):
    edge = edge_path[ -1 ] # Choose the latest edge selected
    if edge != None: # Check game if ended | Also means no more edges
        value, move_probabilities = encoder.callNeuralNetwork( board, alphaZeroNet ) 
        """
        Value: The value of this position
        Move_probabilities: Probability distribution across legal moves
    
        callNeuralNetwork:
            Call the neural network on the given position, get the outputs.
                1. Encodes Position but for inference
                2. Calls the neural network (position, mask) receive (value, policy)
                3. Decode policy output
        
        This is to assign to the expanded node.
        """
        
        new_Q = value / 2. + 0.5 # Calculate the prob | IDK why its like this yet. Gotta see.
        print(board)
        edge.expand( board, new_Q, move_probabilities )
        """
        This function adds a `Node` to the `edge`s child .
        """
        new_Q = 1. - new_Q # The probability of winning according to neural network
    else:
        winner = encoder.parseResult( board.result() )
        if not board.turn:
            winner *= -1
            new_Q = float( winner ) / 2. + 0.5
    
    return new_Q, edge

# Phase 4 | Backpropogation
def backprop(edge_path, node_path, new_Q):
    last_node_idx = len( node_path ) - 1

    for i in range( last_node_idx, -1, -1 ):
        node = node_path[ i ] # Goes through visited nodes
        node.N += 1 # Add visit count
        if ( last_node_idx - i ) % 2 == 0:
            node.sum_Q += new_Q
        else:  
            node.sum_Q += 1. - new_Q
    
    for edge in edge_path:
        if edge != None:
           edge.clearVirtualLoss()

board = chess.Board()
root = MCTS.Root(board, alphaZeroNet)
node_path, edge_path = selection(board)
new_Q, edge = expansion(edge_path)

last_node_idx = len( node_path ) - 1
for i in range( last_node_idx, -1, -1 ):
    node = node_path[ i ] # Goes through visited nodes
    node.N += 1 # Add visit count
    if ( last_node_idx - i ) % 2 == 0:
        print("last node detected", last_node_idx)
        node.sum_Q += new_Q
    else:  
        node.sum_Q += 1. - new_Q

for edge in edge_path:
    if edge != None:
        edge.clearVirtualLoss()

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
last node detected 0


In [12]:
import math

def calcUCT( edge, N_p ):
    """
    Calculate the UCT formula.

    Args:
        edge (Edge) the edge which the UCT formula is for
        N_p (float) the parents visit count

    Returns:
        (float) the calculated value
    """

    Q = edge.getQ()
    N_c = edge.getN()
    P = edge.getP()

    C = 1.5
    print(edge.getMove())

    UCT = Q + P * C * math.sqrt( N_p ) / ( 1 + N_c )
    assert not math.isnan( UCT ), 'Q {} N_c {} P {}'.format( Q, N_c, P )
    
    return UCT

board = chess.Board()
root = MCTS.Root(board, alphaZeroNet)
root.edges[15].addVirtualLoss()

UCT = calcUCT(root.edges[15], root.N)
print(UCT)

e2e4
0.18784521520137787


In [49]:
with torch.no_grad():
    root = Root(board, alphaZeroNet)
    for _ in range(2):
        root.parallelRollouts(board, alphaZeroNet, 15)
    print(root.getStatisticsString()[0])

|   move   |    P     |    N     |    Q     |   UCT    |
|   g1f3   |    0.8549|   16.0000|    0.5540|    0.9739|
|   a2a4   |    0.0000|    0.0000|    0.0000|    0.0000|
|   b2b4   |    0.0002|    0.0000|    0.0000|    0.0019|
|   c2c4   |    0.0036|    0.0000|    0.0000|    0.0299|
|   d2d4   |    0.0096|    0.0000|    0.0000|    0.0804|
|   f2f4   |    0.0186|    0.0000|    0.0000|    0.1556|
|   g2g4   |    0.0000|    0.0000|    0.0000|    0.0001|
|   h2h4   |    0.0000|    0.0000|    0.0000|    0.0002|
|   a2a3   |    0.0012|    0.0000|    0.0000|    0.0100|
|   b2b3   |    0.0031|    0.0000|    0.0000|    0.0260|
|   c2c3   |    0.0208|    0.0000|    0.0000|    0.1733|
|   d2d3   |    0.0029|    0.0000|    0.0000|    0.0243|
|   f2f3   |    0.0002|    0.0000|    0.0000|    0.0015|
|   g2g3   |    0.0050|    0.0000|    0.0000|    0.0421|
|   h2h3   |    0.0001|    0.0000|    0.0000|    0.0007|
|   e4e5   |    0.0000|    0.0000|    0.0000|    0.0002|
|   b1a3   |    0.0007|    0.00

In [52]:
board = chess.Board() # Chess board
root = Root(board, alphaZeroNet) # Initiating the first node
node_path = [] # List of nodes traversed 
edge_path = [] # List of edges traversed

def selection(board, root):
    with torch.no_grad():
        root.selectTask(board, node_path, edge_path)

selection(board, root)

print(node_path, edge_path)

[<__main__.Root object at 0x7f5511d17820>] [<__main__.Edge object at 0x7f5511cefa30>]


In [55]:

print(root.edges[0].has_child())

False


In [76]:

# Phase 3 | Simulation / Rollout

# Working backwards
"""
What value is from the final one?

N

How is this gotten?

Total amount of visits.

How is this gotten?

The AI adds it every time it does a rollover on it. 

Somethings not right. The MCTS does simulations.
But here I don't see them. 
And it seems to be getting the nodes with the max visits?
Why??
"""

In [56]:
print(board)
print(root.getStatisticsString()[0])



r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . P . . .
. . . . . . . .
P P P P . P P P
R N B Q K B N R
|   move   |    P     |    N     |    Q     |   UCT    |
|   e2e4   |    0.3757|    1.0000|    0.0000|    0.2818|
|   a2a4   |    0.0001|    0.0000|    0.0000|    0.0001|
|   b2b4   |    0.0006|    0.0000|    0.0000|    0.0009|
|   c2c4   |    0.2406|    0.0000|    0.0000|    0.3609|
|   d2d4   |    0.1735|    0.0000|    0.0000|    0.2603|
|   f2f4   |    0.0079|    0.0000|    0.0000|    0.0119|
|   g2g4   |    0.0000|    0.0000|    0.0000|    0.0001|
|   h2h4   |    0.0001|    0.0000|    0.0000|    0.0002|
|   a2a3   |    0.0005|    0.0000|    0.0000|    0.0008|
|   b2b3   |    0.0160|    0.0000|    0.0000|    0.0241|
|   c2c3   |    0.0029|    0.0000|    0.0000|    0.0043|
|   d2d3   |    0.0042|    0.0000|    0.0000|    0.0063|
|   e2e3   |    0.0098|    0.0000|    0.0000|    0.0148|
|   f2f3   |    0.0001|    0.0000|    0.0000|    0.0001|
|   g2g3   |    0

In [102]:
print(root.getStatisticsString()[0])


|   move   |    P     |    N     |    Q     |   UCT    |
|   e2e4   |    0.3757|    4.0000|    0.2500|    0.5482|
|   c2c4   |    0.2406|    3.0000|    0.3333|    0.5720|
|   d2d4   |    0.1735|    2.0000|    0.5000|    0.7295|
|   a2a4   |    0.0001|    0.0000|    0.0000|    0.0002|
|   b2b4   |    0.0006|    0.0000|    0.0000|    0.0024|
|   f2f4   |    0.0079|    0.0000|    0.0000|    0.0314|
|   g2g4   |    0.0000|    0.0000|    0.0000|    0.0002|
|   h2h4   |    0.0001|    0.0000|    0.0000|    0.0004|
|   a2a3   |    0.0005|    0.0000|    0.0000|    0.0021|
|   b2b3   |    0.0160|    0.0000|    0.0000|    0.0637|
|   c2c3   |    0.0029|    0.0000|    0.0000|    0.0113|
|   d2d3   |    0.0042|    0.0000|    0.0000|    0.0167|
|   e2e3   |    0.0098|    0.0000|    0.0000|    0.0390|
|   f2f3   |    0.0001|    0.0000|    0.0000|    0.0003|
|   g2g3   |    0.0347|    0.0000|    0.0000|    0.1379|
|   h2h3   |    0.0002|    0.0000|    0.0000|    0.0009|
|   b1a3   |    0.0002|    0.00

In [63]:

with torch.no_grad():
    root = MCTS.Root( board, alphaZeroNet )
r = root.rollout(board, alphaZeroNet)

print(board)

r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . p . P . . .
. . . . . N . .
P P . P . P P P
R N B Q K B . R


1.0850912854075432 2.0


In [29]:

def main( modelFile, mode, color, num_rollouts, num_threads, fen, verbose ):
    #prepare neural network
    alphaZeroNet = AlphaZeroNetwork.AlphaZeroNet(10*2, 128*2)
    #toggle for cpu/gpu
    cuda = True
    if cuda:
        weights = torch.load( modelFile )
    else:
        weights = torch.load( modelFile, map_location=torch.device('cpu') )
    alphaZeroNet.load_state_dict( weights )
    if cuda:
        alphaZeroNet = alphaZeroNet.cuda()
    for param in alphaZeroNet.parameters():
        param.requires_grad = False
        
    alphaZeroNet.eval()
    #create chess board object
    if fen:
        board = chess.Board( fen )
    else:
        board = chess.Board()

    #play chess moves
    while True:
        if board.is_game_over():
            #If the game is over, output the winner and wait for user input to continue
            print( 'Game over. Winner: {}'.format(board.result()))
            board.reset_board()
            c = input( 'Enter any key to continue ' )

        #Print the current state of the board
        if board.turn:
            print('White\'s turn')
        else:
            print('Black\'s turn')
        print( board )

        if mode == 'h' and board.turn == color:
            time.sleep(1)
            #If we are in human mode and it is the humans turn, play the move specified from stdin
            move_list = tolist( board.legal_moves )

            idx = -1
            while not (0 <= idx and idx < len( move_list ) ):
                string = input( 'Choose a move ' )
                
                for i, move in enumerate( move_list ):
                    if str( move ) == string:
                        idx = i
                        break
            
            board.push(move_list[idx])

        else:
            # In all other cases the AI selects the next move
            starttime = time.perf_counter()
            with torch.no_grad():
                root = MCTS.Root( board, alphaZeroNet )
                for i in range( num_rollouts ):
                    root.parallelRollouts( board.copy(), alphaZeroNet, num_threads )

            endtime = time.perf_counter()
            elapsed = endtime - starttime
            Q = root.getQ()
            N = root.getN()
            nps = N / elapsed
            same_paths = root.same_paths
    
            if verbose:
                # In verbose mode, print some statistics
                stat, data = root.getStatisticsString() 
                print( stat )
                print( 'total rollouts {} Q {:0.3f} duplicate paths {} elapsed {:0.2f} nps {:0.2f}'.format( int( N ), Q, same_paths, elapsed, nps ) )
            
            # edge = root.maxNSelect()
            # move = edge.getMove()
            # print( 'best move {}'.format( str( move ) ) )
            # board.push(move)

            # """ Overcomplicated? Who knows.
            
            UCT = {x["UCT"]:x["move"] for x in data}
            PROBABILITIES = {x["P"]:x["move"] for x in data}
            G = sorted(PROBABILITIES.keys(), reverse=True) # All probablitilies from high to low

            highest_uct = root.UCTSelect()
            highest_p = root.maxPSelect()

            if highest_p == highest_uct or (highest_p.getP() - G[1]) <= .1:
                bestmove = highest_uct.getMove()
                print( 'best move {}'.format( str( bestmove ) ), highest_p.getP(), G[1])
                board.push( bestmove )
            elif highest_uct != highest_p:
                for n, uct_item in enumerate(sorted(UCT.keys(), reverse=True)): # Go through all UCT from high to low
                    if n == 0: # Pass if it's the highest since it's already been checked
                        pass
                    else:
                        
                        if PROBABILITIES[G[0]] == UCT[uct_item]: # Check if `n` highest UCT is the highest probability
                            print( 'best move {}'.format( str( UCT[uct_item] ) ) )
                            board.push( UCT[uct_item] )

                                
                        # High UCT + High Prob? Go ahead
                        # High UCT but low prob?
                            # Check if next highest UCT is the highest prob
                                # Go ahead
                            # Keep going until its found
            # """

                    

        if mode == 'p':
            #In profile mode, exit after the first move
            break

In [28]:
def parseColor( colorString ):
    """
    Maps 'w' to True and 'b' to False.
    Args:
        colorString (string) a string representing white or black
    """

    if colorString == 'w' or colorString == 'W':
        return True
    elif colorString == 'b' or colorString == 'B':
        return False
    else:
        print( 'Unrecognized argument for color' )
        exit()

In [31]:

main(
    modelFile="AlphaZeroNet_20x256.pt", 
    mode="h", # Human
    color=parseColor("b"), # White
    num_rollouts=1000,
    num_threads=15,
    fen="",
    verbose=True,
)



White's turn
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
|   move   |    P     |    N     |    Q     |   UCT    |
|   d2d4   |    0.1735| 7770.0000|    0.5510|    0.5551|
|   e2e4   |    0.3757| 3924.0000|    0.5371|    0.5547|
|   c2c4   |    0.2406| 1792.0000|    0.5301|    0.5547|
|   g1f3   |    0.1275| 1264.0000|    0.5359|    0.5545|
|   g2g3   |    0.0347|  102.0000|    0.4914|    0.5534|
|   b1c3   |    0.0052|   43.0000|    0.5325|    0.5541|
|   e2e3   |    0.0098|   33.0000|    0.4991|    0.5523|
|   b2b3   |    0.0160|   29.0000|    0.4545|    0.5527|
|   f2f4   |    0.0079|   21.0000|    0.4864|    0.5525|
|   d2d3   |    0.0042|   11.0000|    0.4867|    0.5512|
|   a2a4   |    0.0001|    0.0000|    0.0000|    0.0111|
|   b2b4   |    0.0006|    0.0000|    0.0000|    0.1129|
|   g2g4   |    0.0000|    0.0000|    0.0000|    0.0082|
|   h2h4   |    0.0001|    0.0000|    0.0000|    0.0191|
|   